In [1]:
import random
import nufhe

size = 8
bits1 = [random.choice([False, True]) for i in range(size)]
bits2 = [random.choice([False, True]) for i in range(size)]
reference = [not (b1 and b2) for b1, b2 in zip(bits1, bits2)]

ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()

ciphertext1 = ctx.encrypt(secret_key, bits1)
ciphertext2 = ctx.encrypt(secret_key, bits2)

vm = ctx.make_virtual_machine(cloud_key)
result = vm.gate_nand(ciphertext1, ciphertext2)
print(type(result))
result_bits = ctx.decrypt(secret_key, result)



/home/pradeep/anaconda3/lib/python3.11/site-packages/pytools/persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


<class 'nufhe.lwe.LweSampleArray'>


In [2]:
import numpy
print(numpy.version.version)

1.26.0


In [3]:
!nvidia-smi

Sat Jan 18 11:04:13 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P0    27W /  N/A |    556MiB /  7982MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install ipython-autotime
%load_ext autotime

time: 278 µs (started: 2025-01-18 11:04:15 +05:30)


In [5]:
#!pip install nufhe[pycuda]
#!pip install pycuda

time: 443 µs (started: 2025-01-18 11:04:15 +05:30)


In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

time: 158 µs (started: 2025-01-18 11:04:16 +05:30)


In [7]:
import time
import random
import nufhe
def add(ciX, ciY):
# fixed iteration since
    for i in range(size):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        ciX = aShiftTemp
        ciY = b

    return b
def add_approx(ciX, ciY):
# fixed iteration since
    for i in range(size):
        if i>4:
            a = vm.gate_and(ciX, ciY)
        else:    
            a=ciX
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        ciX = aShiftTemp
        ciY = b

    return b    

def boolListToInt(bitlists):
    out = 0
    bitlists[:] = bitlists[::-1]
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def fixSizeBoolList(decimal,size):
    x = [[int(x)] for x in bin(decimal)[2:]]
    for i in range(len(x)):
        x[i] = list(map(bool, x[i]))
    x = [[False]]*(size - len(x)) + x
    x.reverse()
    return x
size = 16
#bits1 = [random.choice([False, True]) for i in range(size)]
#bits2 = [random.choice([False, True]) for i in range(size)]
bits1 = fixSizeBoolList(10,size)
bits2 = fixSizeBoolList(2,size)
s=[ [False] for i in range(size)]
reference = [not (b1 and b2) for b1, b2 in zip(bits1, bits2)]

ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()

ciphertext1 = ctx.encrypt(secret_key, bits1)
ciphertext2 = ctx.encrypt(secret_key, bits2)
c_s=ctx.encrypt(secret_key, s)

vm = ctx.make_virtual_machine(cloud_key)
print(bits1)
print(bits2)
c_s[0]=ciphertext2[0]
import time

start = time.time()
result = add(ciphertext1, ciphertext2)
#end = timer()
#print("adder timing",end - start)
print("=========")
print(s)
print("type iss",type(c_s[0]))
print("=========================")
#result=vm.gate_mux(c_s[0],ciphertext1, ciphertext2)
result_bits = ctx.decrypt(secret_key, result)

print(result_bits)
print("result is",boolListToInt(result_bits))

[[False], [True], [False], [True], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False]]
[[False], [True], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False]]
[[False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False], [False]]
type iss <class 'nufhe.lwe.LweSampleArray'>
[[False]
 [False]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]
result is [12]
time: 3.85 s (started: 2025-01-18 11:04:31 +05:30)


In [8]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
def cba4(r,a,b,carry):
    temp= vm.gate_and(a, b) #ab
    temp1= vm.gate_and( b,carry ) #bc
    temp2= vm.gate_and(a, carry) # ac
    temp3= vm.gate_or(temp, temp1) #ab+ac
    r[1]=vm.gate_or(temp3,temp2) # ab+abc+ac
    r[0]=vm.gate_not(r[1]) #(ab+bc+ca)'
    return r
def cba1(r,a,b,carry):
    temp= vm.gate_and(b,carry) #temp=bc
    r[1]=vm.gate_or(temp, a)   #r1=bc+a
    r[0]=vm.gate_not(r[1])     #r0=(bc+a)'
    return r

def threetotwo(r,p0,p1,p2):
    r[0] = vm.gate_or(p0, p1)
    temp=vm.gate_and(p0, p1)
    r[1]=vm.gate_or(temp, p2)
    return r
def twotoone(r, p0,p1 ):
    r[0] = vm.gate_or(p0, p1)
    return r
def is_equal(a,b,nBits):
    #print("type of a",type(a))
    #print("type of b",type(b))
    t2= ctx.encrypt(secret_key, [False])
    #print("length of list",len(a))
    for i in range(0,nBits):
        t1=vm.gate_xor(a[i], b[i])
        t3=vm.gate_or(t2, t1)
        t2=vm.gate_copy(t3)
        equal=vm.gate_not(t3)

    return equal
def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''


    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    onep[0]=one
    testone= [[vm.empty_ciphertext((1,))] for i in range(nbits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nbits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def nonRestoringDivision(m, q,nBits,zero_n ): 
    A=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    temp=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    mux1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    mux2=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    neg2=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    q_c=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    mcopy=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    onep=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    fullzero=ctx.encrypt(secret_key, [[False] for _ in range(size)])
    one= ctx.encrypt(secret_key, [True])
    zero= ctx.encrypt(secret_key, [False])
    signbit= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    #tw= ctx.encrypt(secret_key, [False])
    '''for i in range(len(q)):
        q_c=nufhe.LweSampleArray.copy(q[i])'''
    onep[0]=vm.gate_copy(one)  
    c_zero= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    testone= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    ones= ctx.encrypt(secret_key, [[False] for _ in range(size)])
    
    c_zero=zero
    #A[i]=vm.gate_copy(zero)
    testone=onep
    q_c=q
    #A[i]=zero
    A=fullzero
    mcopy=m
    temp=c_zero
    mux1=fullzero
    equal_res= ctx.encrypt(secret_key, [False])
    
    #temp=m
    ones=vm.gate_not(m)
    '''result=ctx.decrypt(secret_key,ones)
    val=listToString(result)
    print(val)
    print("ones comlplement ",two2dec(val[::-1]) )'''
    neg2=add(ones,onep)
    '''result=ctx.decrypt(secret_key,onep)
    val=listToString(result)
    print(val)
    print("onep is",two2dec(val[::-1]) )'''
    '''result=ctx.decrypt(secret_key,neg2)
    val=listToString(result)
    print(val)
    print("neg2 value ",two2dec(val[::-1]))'''
    
    
    count=nBits
    #print("********* loop started **************")
    
    while (True): 
        #print("===loop==",count)
        for i in range(nBits):
            signbit[i]=A[nBits-1]
        
        '''signplain=[]
        signplain.append([ctx.decrypt(secret_key,signbit)])
        print("signbit is",signplain)'''
        for i in range(nBits-1,0,-1):
            A[i] =A[i-1]    #a=a>>1
        A[0]=q_c[nBits-1]             #a[0]=q[msb]  
        for i in range(nBits-1,0,-1):
            q_c[i] =q_c[i-1]              #q=q>>1 
        '''result=ctx.decrypt(secret_key,A)
        val=listToString(result)
        print(val)
        print("A after left shitf ",two2dec(val[::-1]) )'''
        #aminusm=subtract(A,m,nBits)
        #aplusm=addNumbers(A,m,nBits)
        
        mux1=mux(signbit,mcopy,neg2)   #mux1= mcopy,neg2
        '''result=ctx.decrypt(secret_key,mux1)
        val=listToString(result)
        print(val)
        print("mux1 ",two2dec(val[::-1]) )'''
        result=ctx.decrypt(secret_key,A)
        '''val=listToString(result)
        print(val)
        print("a before add ",two2dec(val[::-1]) )
        result=ctx.decrypt(secret_key,mcopy)
        val=listToString(result)
        print("m before add ",two2dec(val[::-1]) )'''
            
        temp1=add(A,mux1)     #a=a+m or a-m
        
        A=temp1
        '''result=ctx.decrypt(secret_key,A)
        val=listToString(result)
        print("a is",val)
        print("A after add or subration",two2dec(val[::-1]) )'''
        
        #A=mux1[:]
        #equal_res=is_equal(A,c_zero,nBits)
        signbit[0]=A[nBits-1]
        '''signplain1=[]
        signplain1.append([ctx.decrypt(secret_key,signbit[0])])
        print("@@@ signbit beforeupdate of q_C",signplain1)'''
        tw1=mux(signbit[0],zero,one)      
        '''signplain2=[]
        signplain2.append([ctx.decrypt(secret_key,tw1)])
        print("@@@ tw1 beforeupdate of q_C",signplain2)'''
        q_c[0]=tw1   #Q[0]=select(A<zero,zero,one);
        #q_c.insert(0,tw1)
        #temp2 = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
        
        '''result=ctx.decrypt(secret_key,A)
        val=listToString(result)
        print("a is ",val)
        print("count is" ,count,"A inside loop",two2dec(val[::-1]))'''
        
        temp2=q_c
        '''result=ctx.decrypt(secret_key,q_c)
        val=listToString(result)
        print("q_c is",val)
        print("count is" ,count,"q_c inside loop",two2dec(val[::-1]) )'''
        
        count -= 1
        if count==0:
            break
    #aminuszero=subtract(A,c_zero,nBits)
    for i in range(nBits):
        signbit[i]=A[nBits-1]
    aminuszero=A
    aplusm =add(A,m)
    
    A=mux(signbit,aplusm,aminuszero)    #A=select(A<zero,A+M,A);
    return q_c
    #return A,q_c 


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
      pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r


def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
      if i>0:
        bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])
      else:
        bitResult = twotoone(bitResult, ctA[i], ctB[i])
        #bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
        #bitResult=cba4(bitResult, ctA[i], ctB[i],carry)
        #Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
        #carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes


def mulNumbers(ctA, ctB, secret_key, input_bits, output_bits):
    result = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    temp = [ctx.encrypt(secret_key, [False]) for _ in
              range(output_bits)]
    # [[vm.empty_ciphertext((1,))] for _ in range(output_bits)]
    # andRes = [[vm.empty_ciphertext((1,))] for _ in range(input_bits)]

    for i in range(input_bits):
        andResLeft = [ctx.encrypt(secret_key, [False]) for _ in
                      range(output_bits)]
        #temp=mux(temp,ctA,ctB[i],size)
        for j in range(input_bits):
            andResLeft[j + i] = vm.gate_and(ctA[j], ctB[i])

            # andResLeft[j + i] = nufhe.LweSampleArray.copy(andRes[j])
        result = addNumbers(andResLeft, result, output_bits)
        #result_bits = [ctx.decrypt(secret_key, result[i]) for i in range(size * 2)]
        #print(" nuFHE multiplication intermdiate number is : ",i,boolListToInt(result_bits))

    return result[:size]
def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])

    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}"
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s):
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')

    #print(listp)
    str1 = ""
    # traverse in the string
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s:
        str1 += ele
    # return string
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)


    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val
def compare_bit(  a, b,  lsb_carry,  tmp):
  result= ctx.encrypt(secret_key, [False])
  tmp=vm.gate_xnor(a, b)
  result=vm.gate_mux(tmp,lsb_carry, a)
  return result
def minimum(  a,  b,  nb_bits):
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
      tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1
def predict(ctA,onep,onen):
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    for i in range(size):
        c_s[i]=ctA[size-1]
        
    res= vm.gate_mux(c_s,onen,onep)
    return res

def mul(ciX,ciY,enc_temp):

    msb_temp= [vm.empty_ciphertext((1,))]
    control=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    c_s1=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    msb_temp=vm.gate_xor(ciX[size-1],ciY[size-1])
    t=size-1
    for i in range(size):
            control[i]=ciX[t]
            c_s1[i]=ciY[t]
    mul_temp=ciX
    zero=enc_temp
    one_ciX=vm.gate_not(ciX)
    result=ctx.decrypt(secret_key, one_ciX)
    one_ciY=vm.gate_not(ciY)
    comp_ciX=add(one_ciX,one)
    comp_ciY=add(one_ciY,one)

    #print("cix type after for loop",type(ciX))
    #print("control bit:",ctx.decrypt(secret_key, control))
    ciX=mux(control,comp_ciX,ciX)
    ciY=mux(c_s1,comp_ciY,ciY)
    #print("cix:",ctx.decrypt(secret_key, ciX))
    for j in range(0,size):
        #print("=================iteration==========",j)


        #print("cix:",ctx.decrypt(secret_key, ciX))
        for i in range(size):
            c_s[i]=ciY[j]
        add_res=mux(c_s,ciX,zero)

        #enc_theta=add(enc_temp,add_res)
        enc_theta=add_approx(enc_temp,add_res)   #approximate multiplier
        
        enc_temp=enc_theta
        #enc_temp=ciX
        #print("enc_teta:",ctx.decrypt(secret_key, enc_theta))

        #print("control bit:",ctx.decrypt(secret_key, c_s[0]))
        #mux_result=mux(c_s,enc_theta,ciX)
        #mux_result=vm.gate_mux(c_s, enc_theta,ciX)
        #print("mux result:",ctx.decrypt(secret_key, mux_result))
        #ciX=mux_result
        ciX.roll(1, axis=0)
        #ciX.roll(1, axis=0)
    for i in range(size):
        c_s[i]=msb_temp
    one_presult=vm.gate_not(enc_theta)
    comp_presult=add(one_presult,one)
    ciX=vm.gate_mux(c_s,comp_presult,enc_theta)
    #print("cix type is in return",type(ciX))
    return ciX
def mux(c_s,true,false) :
    temp1=vm.gate_and(c_s,true)
    #print("temp1:",ctx.decrypt(secret_key, temp1))
    temp2=vm.gate_and(vm.gate_not(c_s),false)
    #print("temp2:",ctx.decrypt(secret_key, temp2))
    mux_result=vm.gate_or(temp1,temp2)
    return mux_result
def add(ciX, ciY):
# fixed iteration since

    for i in range(size):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        ciY = b
    #result=ctx.decrypt(secret_key,b)
    #print(result)
    #print("b value in add module ",boolListToInt(result))
    return b
def two2dec(s):
    if s[0] == '1':
        return -1 * (int(''.join('1' if x == '0' else '0' for x in s), 2) + 1)
    else:
        return int(s, 2)

time: 6.99 ms (started: 2025-01-18 11:04:35 +05:30)


In [9]:
def nnmult(input,w_ciphertext2d,w_ciphertext2d_bias,vec1,size,enc_temp):
  for i in range(len(w_ciphertext2d[0])):
    print("i count in nnmult",i)
    vec1[i]=nnmult1(input,w_ciphertext2d[i],w_ciphertext2d_bias[i],vec1,size,enc_temp)

  return vec1

def nnmult1(input,w_ciphertext1d_2,w_ciphertext1d_bias_2,vec2,size,enc_temp):
  print("length of cipher text",len(w_ciphertext1d_2))
  for i in range(len(w_ciphertext1d_2)):
    print("i value in nnmult1",i)
    temp=mul(w_ciphertext1d_2[i],input[i],enc_temp)
    print("multiplication completed")
    temp1=add(temp,vec2)
    vec2=temp1
  enc_temp=add(vec2,w_ciphertext1d_bias_2)
  vec2=enc_temp
  return vec2

time: 857 µs (started: 2025-01-18 11:04:35 +05:30)


In [10]:

layer1 = np.load('/home/pradeep/Downloads/layer_0_weights.npy')
layer1_bias=np.load('/home/pradeep/Downloads/layer_1_weights.npy')
layer2 = np.load('/home/pradeep/Downloads/layer_2_weights.npy')
layer2_bias=np.load('/home/pradeep/Downloads/layer_3_weights.npy')

tlayer1=layer1.transpose()
print(tlayer1)
#print(layer2)
#print(layer1_bias)
#print(layer2_bias)
print(np.size(layer1)//np.size(layer1[0]))
print(np.size(layer1[0]))
print(np.size(layer2)//np.size(layer2[0]))
#for i in range(len(layer1)):
 #   for j in range(len(layer1[i])):
 #       print(f"Element at ({i}, {j}): {matrix[i][j]}")'''

[[ 11   0  -3   1   2  -4   1  -1   1  -3   4]
 [  2   1   1   2   1  -1   0   3   0   1   6]
 [ -1  -1   3   6   2   5   0  -2  -2   3  -3]
 [ 14 -12   3   2  -3   2   4  -5   0  -1   0]
 [ -8   7  -3  -3   2   0   0   2   2  -3  -1]
 [  2  -2   2  -3   7  -3   3   2  -3  -2   5]
 [  5 -10  -3   2  -4  -1  -4   0  -2   8  -2]
 [-10  12   0   0   4  -1   0  -1   0  -4   2]]
11
8
8
time: 3.24 ms (started: 2025-01-18 11:04:36 +05:30)


In [11]:
print(type(layer2))
for i in range(len(layer2)):
  print(layer2[i][0])

<class 'numpy.ndarray'>
-9
4
6
-6
8
1
-5
8
time: 327 µs (started: 2025-01-18 11:04:37 +05:30)


In [12]:
import numpy as np
if __name__ == '__main__':
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    n=twos_complement(-1,size)
    p=twos_complement(1,size)
    onen=ctx.encrypt(secret_key, n)
    onep=ctx.encrypt(secret_key, p)
    eten=ctx.encrypt(secret_key, twos_complement(10,size))
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=one
    with open('/home/pradeep/Downloads/pc11train.txt') as f:
      lines = []
      for line in f:
        lines.append([int(v) for v in line.split()])
    temp=fixSizeBoolList(0,size)
    enc_temp=ctx.encrypt(secret_key, temp)
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    layer1 = np.load('/home/pradeep/Downloads/layer_0_weights.npy')
    layer1_bias=np.load('/home/pradeep/Downloads/layer_1_weights.npy')
    layer2 = np.load('/home/pradeep/Downloads/layer_2_weights.npy')
    layer2_bias=np.load('/home/pradeep/Downloads/layer_3_weights.npy')
    layer1=layer1.transpose()

time: 1.23 s (started: 2025-01-18 11:04:39 +05:30)


In [13]:
x0 = twos_complement(int(lines[0][0]), size)
print(x0)
x1 = twos_complement(int(lines[0][1]), size)
x2 = twos_complement(int(lines[0][2]), size)
x3 = twos_complement(int(lines[0][3]), size)
x4 = twos_complement(int(lines[0][4]), size)
x5 = twos_complement(int(lines[0][5]), size)
x6 = twos_complement(int(lines[0][6]), size)
x7 = twos_complement(int(lines[0][7]), size)
x8 = twos_complement(int(lines[0][8]), size)
x9 = twos_complement(int(lines[0][9]), size)
x10 = twos_complement(int(lines[0][10]), size)

# Weights for 8 hidden layer neurons (W_h_i_j represents weight from input j to hidden i)
W_h1_0 = twos_complement(layer1[0][0], size)
W_h1_1 = twos_complement(layer1[0][1], size)
W_h1_2 = twos_complement(layer1[0][2], size)
W_h1_3 = twos_complement(layer1[0][3], size)
W_h1_4 = twos_complement(layer1[0][4], size)
W_h1_5 = twos_complement(layer1[0][5], size)
W_h1_6 = twos_complement(layer1[0][6], size)
W_h1_7 = twos_complement(layer1[0][7], size)
W_h1_8 = twos_complement(layer1[0][8], size)
W_h1_9 = twos_complement(layer1[0][9], size)
W_h1_10 = twos_complement(layer1[0][10], size)

W_h2_0 = twos_complement(layer1[1][0], size)
W_h2_1 = twos_complement(layer1[1][1], size)
W_h2_2 = twos_complement(layer1[1][2], size)
W_h2_3 = twos_complement(layer1[1][3], size)
W_h2_4 = twos_complement(layer1[1][4], size)
W_h2_5 = twos_complement(layer1[1][5], size)
W_h2_6 = twos_complement(layer1[1][6], size)
W_h2_7 = twos_complement(layer1[1][7], size)
W_h2_8 = twos_complement(layer1[1][8], size)
W_h2_9 = twos_complement(layer1[1][9], size)
W_h2_10 = twos_complement(layer1[1][10], size)

W_h3_0 = twos_complement(layer1[2][0], size)
W_h3_1 = twos_complement(layer1[2][1], size)
W_h3_2 = twos_complement(layer1[2][2], size)
W_h3_3 = twos_complement(layer1[2][3], size)
W_h3_4 = twos_complement(layer1[2][4], size)
W_h3_5 = twos_complement(layer1[2][5], size)
W_h3_6 = twos_complement(layer1[2][6], size)
W_h3_7 = twos_complement(layer1[2][7], size)
W_h3_8 = twos_complement(layer1[2][8], size)
W_h3_9 = twos_complement(layer1[2][9], size)
W_h3_10 = twos_complement(layer1[2][10], size)

W_h4_0 = twos_complement(layer1[3][0], size)
W_h4_1 = twos_complement(layer1[3][1], size)
W_h4_2 = twos_complement(layer1[3][2], size)
W_h4_3 = twos_complement(layer1[3][3], size)
W_h4_4 = twos_complement(layer1[3][4], size)
W_h4_5 = twos_complement(layer1[3][5], size)
W_h4_6 = twos_complement(layer1[3][6], size)
W_h4_7 = twos_complement(layer1[3][7], size)
W_h4_8 = twos_complement(layer1[3][8], size)
W_h4_9 = twos_complement(layer1[3][9], size)
W_h4_10 = twos_complement(layer1[3][10], size)

W_h5_0 = twos_complement(layer1[4][0], size)
W_h5_1 = twos_complement(layer1[4][1], size)
W_h5_2 = twos_complement(layer1[4][2], size)
W_h5_3 = twos_complement(layer1[4][3], size)
W_h5_4 = twos_complement(layer1[4][4], size)
W_h5_5 = twos_complement(layer1[4][5], size)
W_h5_6 = twos_complement(layer1[4][6], size)
W_h5_7 = twos_complement(layer1[4][7], size)
W_h5_8 = twos_complement(layer1[4][8], size)
W_h5_9 = twos_complement(layer1[4][9], size)
W_h5_10 = twos_complement(layer1[4][10], size)

W_h6_0 = twos_complement(layer1[5][0], size)
W_h6_1 = twos_complement(layer1[5][1], size)
W_h6_2 = twos_complement(layer1[5][2], size)
W_h6_3 = twos_complement(layer1[5][3], size)
W_h6_4 = twos_complement(layer1[5][4], size)
W_h6_5 = twos_complement(layer1[5][5], size)
W_h6_6 = twos_complement(layer1[5][6], size)
W_h6_7 = twos_complement(layer1[5][7], size)
W_h6_8 = twos_complement(layer1[5][8], size)
W_h6_9 = twos_complement(layer1[5][9], size)
W_h6_10 = twos_complement(layer1[5][10], size)

W_h7_0 = twos_complement(layer1[6][0], size)
W_h7_1 = twos_complement(layer1[6][1], size)
W_h7_2 = twos_complement(layer1[6][2], size)
W_h7_3 = twos_complement(layer1[6][3], size)
W_h7_4 = twos_complement(layer1[6][4], size)
W_h7_5 = twos_complement(layer1[6][5], size)
W_h7_6 = twos_complement(layer1[6][6], size)
W_h7_7 = twos_complement(layer1[6][7], size)
W_h7_8 = twos_complement(layer1[6][8], size)
W_h7_9 = twos_complement(layer1[6][9], size)
W_h7_10 = twos_complement(layer1[6][10], size)

W_h8_0 = twos_complement(layer1[7][0], size)
W_h8_1 = twos_complement(layer1[7][1], size)
W_h8_2 = twos_complement(layer1[7][2], size)
W_h8_3 = twos_complement(layer1[7][3], size)
W_h8_4 = twos_complement(layer1[7][4], size)
W_h8_5 = twos_complement(layer1[7][5], size)
W_h8_6 = twos_complement(layer1[7][6], size)
W_h8_7 = twos_complement(layer1[7][7], size)
W_h8_8 = twos_complement(layer1[7][8], size)
W_h8_9 = twos_complement(layer1[7][9], size)
W_h8_10 = twos_complement(layer1[7][10], size)

# Biases for the hidden layer neurons
bias_h1 = twos_complement(layer1_bias[0], size)
bias_h2 = twos_complement(layer1_bias[1], size)
bias_h3 = twos_complement(layer1_bias[2], size)
bias_h4 = twos_complement(layer1_bias[3], size)
bias_h5 = twos_complement(layer1_bias[4], size)
bias_h6 = twos_complement(layer1_bias[5], size)
bias_h7 = twos_complement(layer1_bias[6], size)
bias_h8 = twos_complement(layer1_bias[7], size)

# Encryption of inputs
x0_encrypted = ctx.encrypt(secret_key, x0)
x1_encrypted = ctx.encrypt(secret_key, x1)
x2_encrypted = ctx.encrypt(secret_key, x2)
x3_encrypted = ctx.encrypt(secret_key, x3)
x4_encrypted = ctx.encrypt(secret_key, x4)
x5_encrypted = ctx.encrypt(secret_key, x5)
x6_encrypted = ctx.encrypt(secret_key, x6)
x7_encrypted = ctx.encrypt(secret_key, x7)
x8_encrypted = ctx.encrypt(secret_key, x8)
x9_encrypted = ctx.encrypt(secret_key, x9)
x10_encrypted = ctx.encrypt(secret_key, x10)

# Encryption of weights
W_h1_0_encrypted = ctx.encrypt(secret_key, W_h1_0)
W_h1_1_encrypted = ctx.encrypt(secret_key, W_h1_1)
W_h1_2_encrypted = ctx.encrypt(secret_key, W_h1_2)
W_h1_3_encrypted = ctx.encrypt(secret_key, W_h1_3)
W_h1_4_encrypted = ctx.encrypt(secret_key, W_h1_4)
W_h1_5_encrypted = ctx.encrypt(secret_key, W_h1_5)
W_h1_6_encrypted = ctx.encrypt(secret_key, W_h1_6)
W_h1_7_encrypted = ctx.encrypt(secret_key, W_h1_7)
W_h1_8_encrypted = ctx.encrypt(secret_key, W_h1_8)
W_h1_9_encrypted = ctx.encrypt(secret_key, W_h1_9)
W_h1_10_encrypted = ctx.encrypt(secret_key, W_h1_10)

W_h2_0_encrypted = ctx.encrypt(secret_key, W_h2_0)
W_h2_1_encrypted = ctx.encrypt(secret_key, W_h2_1)
W_h2_2_encrypted = ctx.encrypt(secret_key, W_h2_2)
W_h2_3_encrypted = ctx.encrypt(secret_key, W_h2_3)
W_h2_4_encrypted = ctx.encrypt(secret_key, W_h2_4)
W_h2_5_encrypted = ctx.encrypt(secret_key, W_h2_5)
W_h2_6_encrypted = ctx.encrypt(secret_key, W_h2_6)
W_h2_7_encrypted = ctx.encrypt(secret_key, W_h2_7)
W_h2_8_encrypted = ctx.encrypt(secret_key, W_h2_8)
W_h2_9_encrypted = ctx.encrypt(secret_key, W_h2_9)
W_h2_10_encrypted = ctx.encrypt(secret_key, W_h2_10)

W_h3_0_encrypted = ctx.encrypt(secret_key, W_h3_0)
W_h3_1_encrypted = ctx.encrypt(secret_key, W_h3_1)
W_h3_2_encrypted = ctx.encrypt(secret_key, W_h3_2)
W_h3_3_encrypted = ctx.encrypt(secret_key, W_h3_3)
W_h3_4_encrypted = ctx.encrypt(secret_key, W_h3_4)
W_h3_5_encrypted = ctx.encrypt(secret_key, W_h3_5)
W_h3_6_encrypted = ctx.encrypt(secret_key, W_h3_6)
W_h3_7_encrypted = ctx.encrypt(secret_key, W_h3_7)
W_h3_8_encrypted = ctx.encrypt(secret_key, W_h3_8)
W_h3_9_encrypted = ctx.encrypt(secret_key, W_h3_9)
W_h3_10_encrypted = ctx.encrypt(secret_key, W_h3_10)

W_h4_0_encrypted = ctx.encrypt(secret_key, W_h4_0)
W_h4_1_encrypted = ctx.encrypt(secret_key, W_h4_1)
W_h4_2_encrypted = ctx.encrypt(secret_key, W_h4_2)
W_h4_3_encrypted = ctx.encrypt(secret_key, W_h4_3)
W_h4_4_encrypted = ctx.encrypt(secret_key, W_h4_4)
W_h4_5_encrypted = ctx.encrypt(secret_key, W_h4_5)
W_h4_6_encrypted = ctx.encrypt(secret_key, W_h4_6)
W_h4_7_encrypted = ctx.encrypt(secret_key, W_h4_7)
W_h4_8_encrypted = ctx.encrypt(secret_key, W_h4_8)
W_h4_9_encrypted = ctx.encrypt(secret_key, W_h4_9)
W_h4_10_encrypted = ctx.encrypt(secret_key, W_h4_10)

W_h5_0_encrypted = ctx.encrypt(secret_key, W_h5_0)
W_h5_1_encrypted = ctx.encrypt(secret_key, W_h5_1)
W_h5_2_encrypted = ctx.encrypt(secret_key, W_h5_2)
W_h5_3_encrypted = ctx.encrypt(secret_key, W_h5_3)
W_h5_4_encrypted = ctx.encrypt(secret_key, W_h5_4)
W_h5_5_encrypted = ctx.encrypt(secret_key, W_h5_5)
W_h5_6_encrypted = ctx.encrypt(secret_key, W_h5_6)
W_h5_7_encrypted = ctx.encrypt(secret_key, W_h5_7)
W_h5_8_encrypted = ctx.encrypt(secret_key, W_h5_8)
W_h5_9_encrypted = ctx.encrypt(secret_key, W_h5_9)
W_h5_10_encrypted = ctx.encrypt(secret_key, W_h5_10)

W_h6_0_encrypted = ctx.encrypt(secret_key, W_h6_0)
W_h6_1_encrypted = ctx.encrypt(secret_key, W_h6_1)
W_h6_2_encrypted = ctx.encrypt(secret_key, W_h6_2)
W_h6_3_encrypted = ctx.encrypt(secret_key, W_h6_3)
W_h6_4_encrypted = ctx.encrypt(secret_key, W_h6_4)
W_h6_5_encrypted = ctx.encrypt(secret_key, W_h6_5)
W_h6_6_encrypted = ctx.encrypt(secret_key, W_h6_6)
W_h6_7_encrypted = ctx.encrypt(secret_key, W_h6_7)
W_h6_8_encrypted = ctx.encrypt(secret_key, W_h6_8)
W_h6_9_encrypted = ctx.encrypt(secret_key, W_h6_9)
W_h6_10_encrypted = ctx.encrypt(secret_key, W_h6_10)

W_h7_0_encrypted = ctx.encrypt(secret_key, W_h7_0)
W_h7_1_encrypted = ctx.encrypt(secret_key, W_h7_1)
W_h7_2_encrypted = ctx.encrypt(secret_key, W_h7_2)
W_h7_3_encrypted = ctx.encrypt(secret_key, W_h7_3)
W_h7_4_encrypted = ctx.encrypt(secret_key, W_h7_4)
W_h7_5_encrypted = ctx.encrypt(secret_key, W_h7_5)
W_h7_6_encrypted = ctx.encrypt(secret_key, W_h7_6)
W_h7_7_encrypted = ctx.encrypt(secret_key, W_h7_7)
W_h7_8_encrypted = ctx.encrypt(secret_key, W_h7_8)
W_h7_9_encrypted = ctx.encrypt(secret_key, W_h7_9)
W_h7_10_encrypted = ctx.encrypt(secret_key, W_h7_10)

W_h8_0_encrypted = ctx.encrypt(secret_key, W_h8_0)
W_h8_1_encrypted = ctx.encrypt(secret_key, W_h8_1)
W_h8_2_encrypted = ctx.encrypt(secret_key, W_h8_2)
W_h8_3_encrypted = ctx.encrypt(secret_key, W_h8_3)
W_h8_4_encrypted = ctx.encrypt(secret_key, W_h8_4)
W_h8_5_encrypted = ctx.encrypt(secret_key, W_h8_5)
W_h8_6_encrypted = ctx.encrypt(secret_key, W_h8_6)
W_h8_7_encrypted = ctx.encrypt(secret_key, W_h8_7)
W_h8_8_encrypted = ctx.encrypt(secret_key, W_h8_8)
W_h8_9_encrypted = ctx.encrypt(secret_key, W_h8_9)
W_h8_10_encrypted = ctx.encrypt(secret_key, W_h8_10)

# Encryption of biases
bias_h1_encrypted = ctx.encrypt(secret_key, bias_h1)
bias_h2_encrypted = ctx.encrypt(secret_key, bias_h2)
bias_h3_encrypted = ctx.encrypt(secret_key, bias_h3)
bias_h4_encrypted = ctx.encrypt(secret_key, bias_h4)
bias_h5_encrypted = ctx.encrypt(secret_key, bias_h5)
bias_h6_encrypted = ctx.encrypt(secret_key, bias_h6)
bias_h7_encrypted = ctx.encrypt(secret_key, bias_h7)
bias_h8_encrypted =ctx.encrypt(secret_key, bias_h8)


out_w_x0 = twos_complement(layer2[0][0], size)
out_w_x1 = twos_complement(layer2[1][0], size)
out_w_x2 = twos_complement(layer2[2][0], size)
out_w_x3 = twos_complement(layer2[3][0], size)
out_w_x4 = twos_complement(layer2[4][0], size)
out_w_x5 = twos_complement(layer2[5][0], size)
out_w_x6 = twos_complement(layer2[6][0], size)
out_w_x7 = twos_complement(layer2[7][0], size)
out_bias_x0 = twos_complement(layer2_bias[0], size)

out_ciphertext1 = ctx.encrypt(secret_key, out_w_x0)
out_ciphertext2 = ctx.encrypt(secret_key, out_w_x1)
out_ciphertext3 = ctx.encrypt(secret_key, out_w_x2)
out_ciphertext4 = ctx.encrypt(secret_key, out_w_x3)
out_ciphertext5 = ctx.encrypt(secret_key, out_w_x4)
out_ciphertext6 = ctx.encrypt(secret_key, out_w_x5)
out_ciphertext7 = ctx.encrypt(secret_key, out_w_x6)
out_ciphertext8 = ctx.encrypt(secret_key, out_w_x7)
out_bias_ciphertext = ctx.encrypt(secret_key, out_bias_x0)

[[True], [True], [True], [False], [True], [False], [True], [True], [True], [True], [True], [True], [True], [True], [True], [True]]
time: 60 ms (started: 2025-01-18 11:04:40 +05:30)


In [14]:

def predict(ctA,onep,onen):
    c_s=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    #print("onep type",type(onep))
    #print("one n type",type(onen))
    for i in range(size):
        c_s[i]=ctA[size-1]

    res= vm.gate_mux(c_s,onen,onep)
    return res

time: 323 µs (started: 2025-01-18 11:04:40 +05:30)


In [15]:
n=twos_complement(-1,size)
p=twos_complement(1,size)
zero=twos_complement(0,size)
onen=ctx.encrypt(secret_key, n)
onep=ctx.encrypt(secret_key, p)
czero=ctx.encrypt(secret_key, zero)

time: 3.03 ms (started: 2025-01-18 11:04:42 +05:30)


In [16]:
#h1_mul0 = mul(x0_encrypted, W_h1_0_encrypted, enc_temp)
#print("type of h_mul0",type(h1_mul0))

time: 148 µs (started: 2025-01-18 11:04:43 +05:30)


In [17]:
# Hidden Neuron 1
h1_mul0 = mul(x0_encrypted, W_h1_0_encrypted, enc_temp)
h1_mul1 = mul(x1_encrypted, W_h1_1_encrypted, enc_temp)
h1_mul2 = mul(x2_encrypted, W_h1_2_encrypted, enc_temp)
h1_mul3 = mul(x3_encrypted, W_h1_3_encrypted, enc_temp)
h1_mul4 = mul(x4_encrypted, W_h1_4_encrypted, enc_temp)
h1_mul5 = mul(x5_encrypted, W_h1_5_encrypted, enc_temp)
h1_mul6 = mul(x6_encrypted, W_h1_6_encrypted, enc_temp)
h1_mul7 = mul(x7_encrypted, W_h1_7_encrypted, enc_temp)
h1_mul8 = mul(x8_encrypted, W_h1_8_encrypted, enc_temp)
h1_mul9 = mul(x9_encrypted, W_h1_9_encrypted, enc_temp)
h1_mul10 = mul(x10_encrypted, W_h1_10_encrypted, enc_temp)

h1_add1 = add(h1_mul0, h1_mul1)
#print("type of h_add1",type(h1_add1))
h1_add2 = add(h1_mul2, h1_mul3)
h1_add3 = add(h1_mul4, h1_mul5)
h1_add4 = add(h1_mul6, h1_mul7)
h1_add5 = add(h1_mul8, h1_mul9)

h1_add6 = add(h1_add1, h1_add2)
h1_add7 = add(h1_add3, h1_add4)
h1_add8 = add(h1_add5, h1_mul10)

h1_sum = add(h1_add6, h1_add7)
h1_sum_final = add(h1_sum, h1_add8)
#print("type of sum ",type(h1_sum_final))
hidden1_output = add(h1_sum_final, bias_h1_encrypted)
#print("type of hidden",type(hidden1_output))
hidden1_output=predict(hidden1_output,hidden1_output,czero)
#print("type of hidden1 after relu",type(hidden1_output))

time: 7min 22s (started: 2025-01-18 11:04:44 +05:30)


In [18]:
# Hidden Neuron 2
h2_mul0 = mul(x0_encrypted, W_h2_0_encrypted, enc_temp)
h2_mul1 = mul(x1_encrypted, W_h2_1_encrypted, enc_temp)
h2_mul2 = mul(x2_encrypted, W_h2_2_encrypted, enc_temp)
h2_mul3 = mul(x3_encrypted, W_h2_3_encrypted, enc_temp)
h2_mul4 = mul(x4_encrypted, W_h2_4_encrypted, enc_temp)
h2_mul5 = mul(x5_encrypted, W_h2_5_encrypted, enc_temp)
h2_mul6 = mul(x6_encrypted, W_h2_6_encrypted, enc_temp)
h2_mul7 = mul(x7_encrypted, W_h2_7_encrypted, enc_temp)
h2_mul8 = mul(x8_encrypted, W_h2_8_encrypted, enc_temp)
h2_mul9 = mul(x9_encrypted, W_h2_9_encrypted, enc_temp)
h2_mul10 = mul(x10_encrypted, W_h2_10_encrypted, enc_temp)

h2_add1 = add(h2_mul0, h2_mul1)
h2_add2 = add(h2_mul2, h2_mul3)
h2_add3 = add(h2_mul4, h2_mul5)
h2_add4 = add(h2_mul6, h2_mul7)
h2_add5 = add(h2_mul8, h2_mul9)

h2_add6 = add(h2_add1, h2_add2)
h2_add7 = add(h2_add3, h2_add4)
h2_add8 = add(h2_add5, h2_mul10)

h2_sum = add(h2_add6, h2_add7)
h2_sum_final = add(h2_sum, h2_add8)
hidden2_output = add(h2_sum_final,bias_h2_encrypted)
hidden2_output=predict(hidden2_output,hidden2_output,czero)

time: 7min 26s (started: 2025-01-18 11:12:06 +05:30)


In [19]:
# Hidden Neuron 3
h3_mul0 = mul(x0_encrypted, W_h3_0_encrypted, enc_temp)
h3_mul1 = mul(x1_encrypted, W_h3_1_encrypted, enc_temp)
h3_mul2 = mul(x2_encrypted, W_h3_2_encrypted, enc_temp)
h3_mul3 = mul(x3_encrypted, W_h3_3_encrypted, enc_temp)
h3_mul4 = mul(x4_encrypted, W_h3_4_encrypted, enc_temp)
h3_mul5 = mul(x5_encrypted, W_h3_5_encrypted, enc_temp)
h3_mul6 = mul(x6_encrypted, W_h3_6_encrypted, enc_temp)
h3_mul7 = mul(x7_encrypted, W_h3_7_encrypted, enc_temp)
h3_mul8 = mul(x8_encrypted, W_h3_8_encrypted, enc_temp)
h3_mul9 = mul(x9_encrypted, W_h3_9_encrypted, enc_temp)
h3_mul10 = mul(x10_encrypted, W_h3_10_encrypted, enc_temp)

h3_add1 = add(h3_mul0, h3_mul1)
h3_add2 = add(h3_mul2, h3_mul3)
h3_add3 = add(h3_mul4, h3_mul5)
h3_add4 = add(h3_mul6, h3_mul7)
h3_add5 = add(h3_mul8, h3_mul9)

h3_add6 = add(h3_add1, h3_add2)
h3_add7 = add(h3_add3, h3_add4)
h3_add8 = add(h3_add5, h3_mul10)

h3_sum = add(h3_add6, h3_add7)
h3_sum_final = add(h3_sum, h3_add8)
hidden3_output = add(h3_sum_final, bias_h3_encrypted)
hidden3_output=predict(hidden3_output,hidden3_output,czero)

time: 7min 24s (started: 2025-01-18 11:19:33 +05:30)


In [20]:
# Hidden Neuron 4
h4_mul0 = mul(x0_encrypted, W_h4_0_encrypted, enc_temp)
h4_mul1 = mul(x1_encrypted, W_h4_1_encrypted, enc_temp)
h4_mul2 = mul(x2_encrypted, W_h4_2_encrypted, enc_temp)
h4_mul3 = mul(x3_encrypted, W_h4_3_encrypted, enc_temp)
h4_mul4 = mul(x4_encrypted, W_h4_4_encrypted, enc_temp)
h4_mul5 = mul(x5_encrypted, W_h4_5_encrypted, enc_temp)
h4_mul6 = mul(x6_encrypted, W_h4_6_encrypted, enc_temp)
h4_mul7 = mul(x7_encrypted, W_h4_7_encrypted, enc_temp)
h4_mul8 = mul(x8_encrypted, W_h4_8_encrypted, enc_temp)
h4_mul9 = mul(x9_encrypted, W_h4_9_encrypted, enc_temp)
h4_mul10 = mul(x10_encrypted, W_h4_10_encrypted, enc_temp)

h4_add1 = add(h4_mul0, h4_mul1)
h4_add2 = add(h4_mul2, h4_mul3)
h4_add3 = add(h4_mul4, h4_mul5)
h4_add4 = add(h4_mul6, h4_mul7)
h4_add5 = add(h4_mul8, h4_mul9)

h4_add6 = add(h4_add1, h4_add2)
h4_add7 = add(h4_add3, h4_add4)
h4_add8 = add(h4_add5, h4_mul10)

h4_sum = add(h4_add6, h4_add7)
h4_sum_final = add(h4_sum, h4_add8)
hidden4_output = add(h4_sum_final, bias_h4_encrypted)
hidden4_output=predict(hidden4_output,hidden4_output,czero)

time: 7min 27s (started: 2025-01-18 11:26:57 +05:30)


In [21]:
# Hidden Neuron 5
h5_mul0 = mul(x0_encrypted, W_h5_0_encrypted, enc_temp)
h5_mul1 = mul(x1_encrypted, W_h5_1_encrypted, enc_temp)
h5_mul2 = mul(x2_encrypted, W_h5_2_encrypted, enc_temp)
h5_mul3 = mul(x3_encrypted, W_h5_3_encrypted, enc_temp)
h5_mul4 = mul(x4_encrypted, W_h5_4_encrypted, enc_temp)
h5_mul5 = mul(x5_encrypted, W_h5_5_encrypted, enc_temp)
h5_mul6 = mul(x6_encrypted, W_h5_6_encrypted, enc_temp)
h5_mul7 = mul(x7_encrypted, W_h5_7_encrypted, enc_temp)
h5_mul8 = mul(x8_encrypted, W_h5_8_encrypted, enc_temp)
h5_mul9 = mul(x9_encrypted, W_h5_9_encrypted, enc_temp)
h5_mul10 = mul(x10_encrypted, W_h5_10_encrypted, enc_temp)

h5_add1 = add(h5_mul0, h5_mul1)
h5_add2 = add(h5_mul2, h5_mul3)
h5_add3 = add(h5_mul4, h5_mul5)
h5_add4 = add(h5_mul6, h5_mul7)
h5_add5 = add(h5_mul8, h5_mul9)

h5_add6 = add(h5_add1, h5_add2)
h5_add7 = add(h5_add3, h5_add4)
h5_add8 = add(h5_add5, h5_mul10)

h5_sum = add(h5_add6, h5_add7)
h5_sum_final = add(h5_sum, h5_add8)
hidden5_output = add(h5_sum_final, bias_h5_encrypted)
hidden5_output=predict(hidden5_output,hidden5_output,czero)

time: 7min 33s (started: 2025-01-18 11:34:25 +05:30)


In [22]:
# Hidden Neuron 6
h6_mul0 = mul(x0_encrypted, W_h6_0_encrypted, enc_temp)
h6_mul1 = mul(x1_encrypted, W_h6_1_encrypted, enc_temp)
h6_mul2 = mul(x2_encrypted, W_h6_2_encrypted, enc_temp)
h6_mul3 = mul(x3_encrypted, W_h6_3_encrypted, enc_temp)
h6_mul4 = mul(x4_encrypted, W_h6_4_encrypted, enc_temp)
h6_mul5 = mul(x5_encrypted, W_h6_5_encrypted, enc_temp)
h6_mul6 = mul(x6_encrypted, W_h6_6_encrypted, enc_temp)
h6_mul7 = mul(x7_encrypted, W_h6_7_encrypted, enc_temp)
h6_mul8 = mul(x8_encrypted, W_h6_8_encrypted, enc_temp)
h6_mul9 = mul(x9_encrypted, W_h6_9_encrypted, enc_temp)
h6_mul10 = mul(x10_encrypted, W_h6_10_encrypted, enc_temp)

h6_add1 = add(h6_mul0, h6_mul1)
h6_add2 = add(h6_mul2, h6_mul3)
h6_add3 = add(h6_mul4, h6_mul5)
h6_add4 = add(h6_mul6, h6_mul7)
h6_add5 = add(h6_mul8, h6_mul9)

h6_add6 = add(h6_add1, h6_add2)
h6_add7 = add(h6_add3, h6_add4)
h6_add8 = add(h6_add5, h6_mul10)

h6_sum = add(h6_add6, h6_add7)
h6_sum_final = add(h6_sum, h6_add8)
hidden6_output = add(h6_sum_final, bias_h6_encrypted)
hidden6_output=predict(hidden6_output,hidden6_output,czero)

time: 7min 28s (started: 2025-01-18 11:41:58 +05:30)


In [23]:
# Hidden Neuron 7
h7_mul0 = mul(x0_encrypted, W_h7_0_encrypted, enc_temp)
h7_mul1 = mul(x1_encrypted, W_h7_1_encrypted, enc_temp)
h7_mul2 = mul(x2_encrypted, W_h7_2_encrypted, enc_temp)
h7_mul3 = mul(x3_encrypted, W_h7_3_encrypted, enc_temp)
h7_mul4 = mul(x4_encrypted, W_h7_4_encrypted, enc_temp)
h7_mul5 = mul(x5_encrypted, W_h7_5_encrypted, enc_temp)
h7_mul6 = mul(x6_encrypted, W_h7_6_encrypted, enc_temp)
h7_mul7 = mul(x7_encrypted, W_h7_7_encrypted, enc_temp)
h7_mul8 = mul(x8_encrypted, W_h7_8_encrypted, enc_temp)
h7_mul9 = mul(x9_encrypted, W_h7_9_encrypted, enc_temp)
h7_mul10 = mul(x10_encrypted, W_h7_10_encrypted, enc_temp)

h7_add1 = add(h7_mul0, h7_mul1)
h7_add2 = add(h7_mul2, h7_mul3)
h7_add3 = add(h7_mul4, h7_mul5)
h7_add4 = add(h7_mul6, h7_mul7)
h7_add5 = add(h7_mul8, h7_mul9)

h7_add6 = add(h7_add1, h7_add2)
h7_add7 = add(h7_add3, h7_add4)
h7_add8 = add(h7_add5, h7_mul10)

h7_sum = add(h7_add6, h7_add7)
h7_sum_final = add(h7_sum, h7_add8)
hidden7_output = add(h7_sum_final, bias_h7_encrypted)
hidden7_output=predict(hidden7_output,hidden7_output,czero)

time: 7min 27s (started: 2025-01-18 11:49:26 +05:30)


In [24]:
# Hidden Neuron 8
h8_mul0 = mul(x0_encrypted, W_h8_0_encrypted, enc_temp)
h8_mul1 = mul(x1_encrypted, W_h8_1_encrypted, enc_temp)
h8_mul2 = mul(x2_encrypted, W_h8_2_encrypted, enc_temp)
h8_mul3 = mul(x3_encrypted, W_h8_3_encrypted, enc_temp)
h8_mul4 = mul(x4_encrypted, W_h8_4_encrypted, enc_temp)
h8_mul5 = mul(x5_encrypted, W_h8_5_encrypted, enc_temp)
h8_mul6 = mul(x6_encrypted, W_h8_6_encrypted, enc_temp)
h8_mul7 = mul(x7_encrypted, W_h8_7_encrypted, enc_temp)
h8_mul8 = mul(x8_encrypted, W_h8_8_encrypted, enc_temp)
h8_mul9 = mul(x9_encrypted, W_h8_9_encrypted, enc_temp)
h8_mul10 = mul(x10_encrypted, W_h8_10_encrypted, enc_temp)

h8_add1 = add(h8_mul0, h8_mul1)
h8_add2 = add(h8_mul2, h8_mul3)
h8_add3 = add(h8_mul4, h8_mul5)
h8_add4 = add(h8_mul6, h8_mul7)
h8_add5 = add(h8_mul8, h8_mul9)

h8_add6 = add(h8_add1, h8_add2)
h8_add7 = add(h8_add3, h8_add4)
h8_add8 = add(h8_add5, h8_mul10)

h8_sum = add(h8_add6, h8_add7)
h8_sum_final = add(h8_sum, h8_add8)
hidden8_output = add(h8_sum_final, bias_h8_encrypted)
hidden8_output=predict(hidden8_output,hidden8_output,czero)


time: 7min 27s (started: 2025-01-18 11:56:54 +05:30)


In [25]:
print(type(out_ciphertext1))
print(type(hidden8_output))

<class 'nufhe.lwe.LweSampleArray'>
<class 'nufhe.lwe.LweSampleArray'>
time: 283 µs (started: 2025-01-18 12:04:21 +05:30)


In [26]:
o_mul1 = mul(hidden1_output, out_ciphertext1, enc_temp)
o_mul2 = mul(hidden2_output, out_ciphertext2, enc_temp)
o_mul3 = mul(hidden3_output, out_ciphertext3, enc_temp)
o_mul4 = mul(hidden4_output, out_ciphertext4, enc_temp)
o_mul5 = mul(hidden5_output, out_ciphertext5, enc_temp)
o_mul6 = mul(hidden6_output, out_ciphertext6, enc_temp)
o_mul7 = mul(hidden7_output, out_ciphertext7, enc_temp)
o_mul8 = mul(hidden8_output, out_ciphertext8, enc_temp)

o_add1 = add(o_mul1, o_mul2)
o_add2 = add(o_mul3, o_mul4)
o_add3 = add(o_mul5, o_mul6)
o_add4 = add(o_mul7, o_mul8)

o_add5 = add(o_add1, o_add2)
o_add6 = add(o_add3, o_add4)

o_sum = add(o_add5, o_add6)
output = add(o_sum,out_bias_ciphertext)

time: 5min 28s (started: 2025-01-18 12:04:21 +05:30)


In [27]:
signbit=output[size-1]
#print("signbit is ",ctx.decrypt(secret_key, signbit))
#print("wx type is",type(wx))
#print("wx inside is",type(wx[0]))
bits1 = [ctx.encrypt(secret_key,[False])] #[[vm.empty_ciphertext((1,))] for i in range(size)]
#bits1=[[vm.empty_ciphertext((1,))] for i in range(1)]
#print("bits1 datatype",type(bits1))
#wxcpy=[[vm.empty_ciphertext((1,))] for i in range(size)]
wxcpy=output

temp=output
wx=vm.gate_not(temp)


wxtwos=add(output,onep)
x=mux(signbit,wxtwos,wxcpy)
m=add(x,onep)
'''temp1=m
temp1.reverse()
result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
pa=listToString(result_bits)
print("m values is",twos_comp_val(int(pa,2),len(pa)))'''
wxten=mul(wxcpy, eten, enc_temp)
'''temp1=wxten[:]
temp1.reverse()
result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
pa=listToString(result_bits)
print("wxtenvalue",twos_comp_val(int(pa,2),len(pa)))'''
sigmoid=nonRestoringDivision(m,wxten,size, enc_temp)
'''temp1=sigmoid[:]
temp1.reverse()
result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
pa=listToString(result_bits)
print("divison result",twos_comp_val(int(pa,2),len(pa)))'''
bits1=sigmoid[i]
sigmoidneg=vm.gate_not(bits1)
sigmoidtwos=add(sigmoidneg,onep)
'''temp1=sigmoidtwos[:]
temp1.reverse()
result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
pa=listToString(result_bits)
print("sigmoid2s",twos_comp_val(int(pa,2),len(pa))) '''
fsigmoid=mux(signbit,sigmoidtwos,sigmoid)
'''temp1=fsigmoid[:]
temp1.reverse()
result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
pa=listToString(result_bits)
#print("fsigmoid",pa)
print("fsigmoid",twos_comp_val(int(pa,2),len(pa)))'''
#print("fsigmoid",len(fsigmoid))
'''temp1=presult_add3[:]
temp1.reverse()
result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
pa=listToString(result_bits)
print("thirdadd",pa)
print("add3",twos_comp_val(int(pa,2),len(pa)))'''
#partial_predict = addNumbers(presult_add3, bias,  size)
#partial_predict.revesrse()
'''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
result_bits.reverse()
pa=listToString(result_bits)
print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''

predict1=predict(fsigmoid,onep,onen)
res=ctx.decrypt(secret_key, predict1)
val=listToString(res)
print("2scomplement to dec ",two2dec(val[::-1]) )
#print(" nuFHE multiplication number is : ", plain_predict[i])
#print("i value is",i, "endtime",datetime.now(),"start time",start_time)
#print("start time is",start_time)
#print("end time",time.time())
#print("endtime",datetime.now(),"start time",start_time,"execution time",time.time()-start_time)

2scomplement to dec  1
time: 1min 30s (started: 2025-01-18 12:09:50 +05:30)


/tmp/ipykernel_6231/3039833492.py:316: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  list1=[int(i)for i in s ]


In [ ]:
import numpy as np
if __name__ == '__main__':
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    n=twos_complement(-1,size)
    p=twos_complement(1,size)
    onen=ctx.encrypt(secret_key, n)
    onep=ctx.encrypt(secret_key, p)
    eten=ctx.encrypt(secret_key, twos_complement(10,size))
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=one
    with open('/home/pradeep/Downloads/pc11train.txt') as f:
      lines = []
      for line in f:
        lines.append([int(v) for v in line.split()])
    temp=fixSizeBoolList(0,size)
    enc_temp=ctx.encrypt(secret_key, temp)
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    layer1 = np.load('/home/pradeep/Downloads/layer_0_weights.npy')
    layer1_bias=np.load('/home/pradeep/Downloads/layer_1_weights.npy')
    layer2 = np.load('/home/pradeep/Downloads/layer_2_weights.npy')
    layer2_bias=np.load('/home/pradeep/Downloads/layer_3_weights.npy')
    layer1=layer1.transpose()
    print(layer1)
    print(layer2)
    print(layer1_bias)
    print(layer2_bias)
    size = 16  # Number of rows
    cols = len(layer1[0])  # Number of columns

    rows = len(layer1)  # Number of rows

    w_ciphertext2d = [[ctx.encrypt(secret_key,twos_complement(0,size)) for x in layer1[0]] for row in layer1]
    w_ciphertext2d_bias = [ctx.encrypt(secret_key,twos_complement(0,size)) for x in layer1_bias]

    for i in range(len(layer1)):
      for j in range(len(layer1[0])):
        temp=twos_complement(layer1[i][j],size)
        w_ciphertext2d[i][j] =ctx.encrypt(secret_key,temp)

    for i in range(len(layer1_bias)):
      temp=twos_complement(layer1_bias[i],size)
      w_ciphertext2d_bias[i] =ctx.encrypt(secret_key,temp)

    rows = len(layer2)  # Number of rows
    cols = len(layer2[0])  # Number of columns
    w_ciphertext1d_2 = [ctx.encrypt(secret_key,twos_complement(0,size)) for x in layer2]
    for i in range(len(layer2)):
      temp=twos_complement(layer2[i][0],size)
      w_ciphertext1d_2[i] =ctx.encrypt(secret_key,temp)
    input = [ctx.encrypt(secret_key,twos_complement(0,size)) for x in lines[0]]
    for i in range(len(lines[0])):
      temp=twos_complement(lines[0][i],size)
      input[i] =ctx.encrypt(secret_key,temp)


    w_ciphertext1d_bias_2 =ctx.encrypt(secret_key,layer2_bias)
    vec1 = [ctx.encrypt(secret_key,twos_complement(0,size)) for x in layer1[0]]
    vec2 = ctx.encrypt(secret_key,twos_complement(0,size))

    presult_mul1 = mul(input[i],w_ciphertext2d[0][0],enc_temp)
    print("starting time",datetime.now())
    start_time = time.time()
    vec1=nnmult(input,w_ciphertext2d,w_ciphertext2d_bias,vec1,size,enc_temp)
    vec2=nnmult1(vec1,w_ciphertext1d_2,w_ciphertext1d_bias_2,vec2,size,enc_temp)



    signbit=vec2[size-1]
    print("signbit is ",ctx.decrypt(secret_key, signbit))
    #print("wx type is",type(wx))
    #print("wx inside is",type(wx[0]))
    bits1 = [ctx.encrypt(secret_key,[False])] #[[vm.empty_ciphertext((1,))] for i in range(size)]
    #bits1=[[vm.empty_ciphertext((1,))] for i in range(1)]
    #print("bits1 datatype",type(bits1))
    #wxcpy=[[vm.empty_ciphertext((1,))] for i in range(size)]
    wxcpy=vec2

    temp=vec2
    wx=vm.gate_not(temp)


    wxtwos=add(vec2,onep,enc_temp)
    x=mux(signbit,wxtwos,wxcpy)
    m=add(x,onep,enc_temp)
    '''temp1=m
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
    pa=listToString(result_bits)
    print("m values is",twos_comp_val(int(pa,2),len(pa)))'''
    wxten=mul(wxcpy, eten, enc_temp)
    '''temp1=wxten[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
    pa=listToString(result_bits)
    print("wxtenvalue",twos_comp_val(int(pa,2),len(pa)))'''
    sigmoid=nonRestoringDivision(m,wxten,size,enc_temp)
    '''temp1=sigmoid[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
    pa=listToString(result_bits)
    print("divison result",twos_comp_val(int(pa,2),len(pa)))'''
    bits1=sigmoid[i]
    sigmoidneg=vm.gate_not(bits1)
    sigmoidtwos=add(sigmoidneg,onep,enc_temp)
    '''temp1=sigmoidtwos[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
    pa=listToString(result_bits)
    print("sigmoid2s",twos_comp_val(int(pa,2),len(pa))) '''
    fsigmoid=mux(signbit,sigmoidtwos,sigmoid)
    '''temp1=fsigmoid[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
    pa=listToString(result_bits)
    #print("fsigmoid",pa)
    print("fsigmoid",twos_comp_val(int(pa,2),len(pa)))'''
    #print("fsigmoid",len(fsigmoid))
    '''temp1=presult_add3[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
    pa=listToString(result_bits)
    print("thirdadd",pa)
    print("add3",twos_comp_val(int(pa,2),len(pa)))'''
    #partial_predict = addNumbers(presult_add3, bias,  size)
    #partial_predict.revesrse()
    '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
    result_bits.reverse()
    pa=listToString(result_bits)
    print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''

    predict1=predict(fsigmoid,onep,onen)
    res=ctx.decrypt(secret_key, predict1)
    val=listToString(res)
    print("2scomplement to dec ",two2dec(val[::-1]) )
    #print(" nuFHE multiplication number is : ", plain_predict[i])
    print("i value is",i, "endtime",datetime.now(),"start time",start_time)
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time,"execution time",time.time()-start_time)
    #result.reverse()
    #for j in range(test_size):
        #print(" nuFHE multiplication number is actual",lines[j][11],"preicted", plain_predict[j])
















In [ ]:
import numpy as np
if __name__ == '__main__':
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    n=twos_complement(-1,size)
    p=twos_complement(1,size)
    onen=ctx.encrypt(secret_key, n)
    onep=ctx.encrypt(secret_key, p)
    eten=ctx.encrypt(secret_key, twos_complement(10,size))
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=one
    with open('/content/drive/MyDrive/pc11train.txt') as f:
      lines = []
      for line in f:
        lines.append([int(v) for v in line.split()])
    temp=fixSizeBoolList(0,size)
    enc_temp=ctx.encrypt(secret_key, temp)
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    layer1 = np.load('/content/drive/MyDrive/layer_0_weights.npy')
    layer1_bias=np.load('/content/drive/MyDrive/layer_1_weights.npy')
    layer2 = np.load('/content/drive/MyDrive/layer_2_weights.npy')
    layer2_bias=np.load('/content/drive/MyDrive/layer_3_weights.npy')
    layer1=layer1.transpose()




w_x0=twos_complement(w_p[0],size)
print("w_x1  is",w_x0)
w_x1=twos_complement(w_p[1],size)

w_x2=twos_complement(w_p[2],size)
w_x3=twos_complement(w_p[3],size)
w_x4=twos_complement(w_p[4],size)
w_x5=twos_complement(w_p[5],size)
print("5 and 6 is",w_p[5],w_p[6])
w_x6=twos_complement(w_p[6],size)
w_x7=twos_complement(w_p[7],size)
w_x8=twos_complement(w_p[8],size)
w_x9=twos_complement(w_p[9],size)
w_x10=twos_complement(w_p[10],size)
bias_x0=twos_complement(w_p[11],size)

w_ciphertext1=ctx.encrypt(secret_key, w_x0)
print("ciy:",ctx.decrypt(secret_key, w_ciphertext1))
w_ciphertext2=ctx.encrypt(secret_key, w_x1)
w_ciphertext3=ctx.encrypt(secret_key, w_x2)
w_ciphertext4=ctx.encrypt(secret_key, w_x3)
w_ciphertext5=ctx.encrypt(secret_key, w_x4)
w_ciphertext6=ctx.encrypt(secret_key, w_x5)
w_ciphertext7=ctx.encrypt(secret_key, w_x6)
w_ciphertext8=ctx.encrypt(secret_key, w_x7)
w_ciphertext9=ctx.encrypt(secret_key, w_x8)
w_ciphertext10=ctx.encrypt(secret_key, w_x9)
w_ciphertext11=ctx.encrypt(secret_key, w_x10)
bias_ciphertext=ctx.encrypt(secret_key, bias_x0)



print(lines[i][0],lines[i][1],lines[i][2],lines[i][3],lines[i][4],lines[i][5],lines[i][6],lines[i][7],lines[i][8],lines[i][9],lines[i][10])
    temp=int(lines[i][0])
    b_x0=twos_complement(temp,size)
    temp=int(lines[i][1])
    b_x1=twos_complement(temp,size)
    temp=int(lines[i][2])
    b_x2=twos_complement(temp,size)
    temp=int(lines[i][3])
    b_x3=twos_complement(temp,size)
    temp=int(lines[i][4])
    b_x4=twos_complement(temp,size)
    temp=int(lines[i][5])
    b_x5=twos_complement(temp,size)
    temp=int(lines[i][6])
    b_x6=twos_complement(temp,size)
    temp=int(lines[i][7])
    b_x7=twos_complement(temp,size)
    temp=int(lines[i][8])
    b_x8=twos_complement(temp,size)
    temp=int(lines[i][9])
    b_x9=twos_complement(temp,size)
    temp=int(lines[i][10])
    b_x10=twos_complement(temp,size)
    ciphertext1=ctx.encrypt(secret_key, b_x0)
    ciphertext2=ctx.encrypt(secret_key, b_x1)
    ciphertext3=ctx.encrypt(secret_key, b_x2)
    ciphertext4=ctx.encrypt(secret_key, b_x3)
    ciphertext5=ctx.encrypt(secret_key, b_x4)
    ciphertext6=ctx.encrypt(secret_key, b_x5)
    ciphertext7=ctx.encrypt(secret_key, b_x6)
    ciphertext8=ctx.encrypt(secret_key, b_x7)
    ciphertext9=ctx.encrypt(secret_key, b_x8)
    ciphertext10=ctx.encrypt(secret_key, b_x9)
    ciphertext11=ctx.encrypt(secret_key, b_x10)
    presult_mul1 = mul(ciphertext1,w_ciphertext1,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul1)
    val=listToString(res)
    print("partialpredict1mul ",two2dec(val[::-1]) )'''
    presult_mul2 = mul(ciphertext2,w_ciphertext2,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul2)
    val=listToString(res)
    print("partialpredict2mul ",two2dec(val[::-1]) )'''
    presult_mul3 = mul(ciphertext3,w_ciphertext3,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul3)
    val=listToString(res)
    print("partialpredict3mul ",two2dec(val[::-1]) )'''
    presult_mul4 = mul(ciphertext4,w_ciphertext4,enc_temp)
    '''res=ctx.decrypt(secret_key,presult_mul4)
    val=listToString(res)
    print("partialpredict4mul ",two2dec(val[::-1]) )'''
    presult_mul5 = mul(ciphertext5,w_ciphertext5,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul5)
    val=listToString(res)
    print("partialpredict5mul ",two2dec(val[::-1]) )'''
    presult_mul6 = mul(ciphertext6,w_ciphertext6,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul6)
    val=listToString(res)
    print("partialpredict6mul ",two2dec(val[::-1]) )'''
    presult_mul7 = mul(ciphertext7,w_ciphertext7,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul7)
    val=listToString(res)
    print("partialpredict7mul ",two2dec(val[::-1]) )'''
    presult_mul8 = mul(ciphertext8,w_ciphertext8,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul8)
    val=listToString(res)
    print("partialpredict8mul ",two2dec(val[::-1]) )'''
    presult_mul9 = mul(ciphertext9,w_ciphertext9,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul9)
    val=listToString(res)
    print("partialpredict9mul ",two2dec(val[::-1]) )'''
    presult_mul10 = mul(ciphertext10,w_ciphertext10,enc_temp)
    '''res=ctx.decrypt(secret_key,presult_mul10)
    val=listToString(res)
    print("partialpredict10mul ",two2dec(val[::-1]) )'''
    presult_mul11 = mul(ciphertext11,w_ciphertext11,enc_temp)
    '''res=ctx.decrypt(secret_key, presult_mul11)
    val=listToString(res)
    print("partialpredict11mul ",two2dec(val[::-1]) )'''
    presult_add1 = add(presult_mul1, presult_mul2)
    '''res=ctx.decrypt(secret_key, presult_add1)
    val=listToString(res)
    print("partialpredictadd1 ",two2dec(val[::-1]) )'''
    presult_add2 = add(presult_mul3, presult_mul4)
    '''res=ctx.decrypt(secret_key, presult_add2)
    val=listToString(res)
    print("partialpredictadd2 ",two2dec(val[::-1]) )'''
    presult_add3 = add(presult_mul5, presult_mul6)
    '''res=ctx.decrypt(secret_key, presult_add3)
    val=listToString(res)
    print("partialpredictadd3 ",two2dec(val[::-1]) )'''
    presult_add4 = add(presult_mul7, presult_mul8)
    '''res=ctx.decrypt(secret_key, presult_add4)
    val=listToString(res)
    print("partialpredictadd4 ",two2dec(val[::-1]) )'''
    presult_add5 = add(presult_mul9, presult_mul10)
    '''res=ctx.decrypt(secret_key, presult_add5)
    val=listToString(res)
    print("partialpredictadd5 ",two2dec(val[::-1]) )'''
    #first step
    presult_add11 = add(presult_add1, presult_add2)
    presult_add12 = add(presult_add3, presult_add4)
    presult_add13 = add(presult_add5, presult_mul11)
    #secong step
    presult_add21 = add(presult_add11, presult_add12)
    presult_add22 = add(presult_add13, bias_ciphertext)
    partial_predict = add(presult_add21,presult_add22)
    res=ctx.decrypt(secret_key, partial_predict)
    val=listToString(res)
    print("partialpredict ",two2dec(val[::-1]) )
    predict1=predict(partial_predict,onep,onen)
    res=ctx.decrypt(secret_key, predict1)
    val=listToString(res)
    print("2scomplement to dec ",two2dec(val[::-1]) )